In [1]:
# Import Splinter and BeautifulSoup
import os
import pandas as pd
import json
import time
import re
from datetime import datetime

In [31]:
def get_file_dates(a):
    strs = re.findall('(\d+)', a)
    if len(strs) >= 1:
        return int(strs[0])
    else:
        return 0

In [32]:
all_files = os.listdir()
filenames = []
for file in all_files:
    if 'player_mapstat' in file:
        filenames.append(file)
filenames = sorted(filenames, key=get_file_dates, reverse=True)
filenames

['player_mapstat_20200617_20200601.csv',
 'player_mapstat_20200617_20200610.csv',
 'player_mapstat_20200614_20200613.csv',
 'player_mapstat_20200614_20200614.csv',
 'player_mapstat_20200610_20200609.csv',
 'player_mapstat_20200609_20200601.csv',
 'player_mapstat_20200601_20200501.csv',
 'player_mapstat_20200601_20200515.csv',
 'player_mapstat_20200515_20200501.csv',
 'player_mapstat_20200507_20200501.csv',
 'player_mapstat_20200507_20200506.csv',
 'player_mapstat_20200501_20200415.csv',
 'player_mapstat_20200415_20200401.csv',
 'player_mapstat_20200402_20200315.csv',
 'player_mapstat_20200316_20200301.csv',
 'player_mapstat_20200302_20200215.csv',
 'player_mapstat_20200215_20200201.csv',
 'player_mapstat_20200201_20200115.csv',
 'player_mapstat_20200115_20200102.csv',
 'player_mapstat_20191231_20190716.csv',
 'player_mapstat_20191231_20191215.csv',
 'player_mapstat_20191215_20191201.csv',
 'player_mapstat_20191205_20191201.csv',
 'player_mapstat_20191201_20191115.csv',
 'player_mapstat

In [7]:
filename = 'player_mapstat_20200617_20200601.csv'

In [87]:
# show the data
df = pd.read_csv(filename)
df

,player_id,player,match_id,date,kills,hs,assists,flash_assists,kdratio,deaths,...,adr,fkdiff,rating,teamname,winner,winner_id,total_rounds,mapstatid,win_rounds,map
16597,16597,4pack,2342227,2020-06-17,31,9,7,2,7,20,...,106,4,1.56,"blood, sweat, and tears","blood,%20sweat,%20and%20tears",10692,30.0,104371,16,Inferno
9572,9572,Marved,2342227,2020-06-17,19,11,9,1,9,20,...,81,-2,1.08,"blood, sweat, and tears","blood,%20sweat,%20and%20tears",10692,30.0,104371,16,Inferno
14131,14131,retchy,2342227,2020-06-17,19,13,2,1,2,19,...,64,-2,1.01,"blood, sweat, and tears","blood,%20sweat,%20and%20tears",10692,30.0,104371,16,Inferno
14109,14109,XotiC,2342227,2020-06-17,22,15,5,3,5,19,...,61,0,0.98,"blood, sweat, and tears","blood,%20sweat,%20and%20tears",10692,30.0,104371,16,Inferno
16649,16649,shanks,2342227,2020-06-17,16,10,6,1,6,19,...,68,-2,0.86,"blood, sweat, and tears","blood,%20sweat,%20and%20tears",10692,30.0,104371,16,Inferno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,2553,Snax,2341793,2020-06-01,25,8,3,0,3,7,...,115,1,2.11,Illuminar,Illuminar,8813,20.0,103464,16,Dust2
3669,3669,innocent,2341793,2020-06-01,16,11,3,3,3,10,...,91,3,1.47,Illuminar,Illuminar,8813,20.0,103464,16,Dust2
8242,8242,reatz,2341793,2020-06-01,17,11,4,2,4,12,...,90,-2,1.35,Illuminar,Illuminar,8813,20.0,103464,16,Dust2
15204,15204,Vegi,2341793,2020-06-01,16,9,5,1,5,14,...,91,1,1.33,Illuminar,Illuminar,8813,20.0,103464,16,Dust2


In [9]:
ddf = df.copy()
ddf['winner'] = list(map(lambda x: x.replace('%20', ' '), ddf['winner']))
ddf

,player_id,player,match_id,date,kills,hs,assists,flash_assists,kdratio,deaths,...,adr,fkdiff,rating,teamname,winner,winner_id,total_rounds,mapstatid,win_rounds,map
16597,16597,4pack,2342227,2020-06-17,31,9,7,2,7,20,...,106,4,1.56,"blood, sweat, and tears","blood, sweat, and tears",10692,30.0,104371,16,Inferno
9572,9572,Marved,2342227,2020-06-17,19,11,9,1,9,20,...,81,-2,1.08,"blood, sweat, and tears","blood, sweat, and tears",10692,30.0,104371,16,Inferno
14131,14131,retchy,2342227,2020-06-17,19,13,2,1,2,19,...,64,-2,1.01,"blood, sweat, and tears","blood, sweat, and tears",10692,30.0,104371,16,Inferno
14109,14109,XotiC,2342227,2020-06-17,22,15,5,3,5,19,...,61,0,0.98,"blood, sweat, and tears","blood, sweat, and tears",10692,30.0,104371,16,Inferno
16649,16649,shanks,2342227,2020-06-17,16,10,6,1,6,19,...,68,-2,0.86,"blood, sweat, and tears","blood, sweat, and tears",10692,30.0,104371,16,Inferno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,2553,Snax,2341793,2020-06-01,25,8,3,0,3,7,...,115,1,2.11,Illuminar,Illuminar,8813,20.0,103464,16,Dust2
3669,3669,innocent,2341793,2020-06-01,16,11,3,3,3,10,...,91,3,1.47,Illuminar,Illuminar,8813,20.0,103464,16,Dust2
8242,8242,reatz,2341793,2020-06-01,17,11,4,2,4,12,...,90,-2,1.35,Illuminar,Illuminar,8813,20.0,103464,16,Dust2
15204,15204,Vegi,2341793,2020-06-01,16,9,5,1,5,14,...,91,1,1.33,Illuminar,Illuminar,8813,20.0,103464,16,Dust2


In [4]:
winner_filename = 'winners.csv'
winner_columns = ['winner', 'winner_id']
all_team_filename = 'all_teams.csv'
team_columns = ['teamname']

In [5]:
def append_info_(df, filename, col):
    # append winners to 'winners.csv'
    if os.path.exists(filename):
        ddf = pd.read_csv(filename)
    else:
        ddf = pd.DataFrame(columns=col)
        
    ddf = pd.concat([ddf, df[col]])
    ddf = ddf.drop_duplicates()
    ddf.to_csv(filename, index=False)
#    df = df[col].drop_duplicates()
#    df.to_csv(filename, mode='a', header=False)

# delete/create csv files 
def create_team_map():
    # 
    df = pd.DataFrame(columns=winner_columns)
    df.to_csv(winner_filename, index=False)
    #
    df = pd.DataFrame(columns=team_columns)
    df.to_csv(all_team_filename, index=False)

# append all winners to 'winners.csv' and 'all_teams.csv'
# append all teams to 'winners.csv' and 'all_teams.csv'
def append_team_map(df):
    # append winners to 'winners.csv'
    append_info_(df, winner_filename, winner_columns)
    # append all teams to 'all_team.csv'
    append_info_(df, all_team_filename, team_columns)

# return teams NOT in winers.
def get_missing_teams():
    # all teams
    if not os.path.exists(all_team_filename):
        return pd.DataFrame(columns=team_columns)
    all_team_df = pd.read_csv(all_team_filename)
    all_team_df = all_team_df.drop_duplicates(ignore_index=True)

    # winners:
    if not os.path.exists(winner_filename):
        return all_team_df[team_columns] 
    winner_df = pd.read_csv(winner_filename)
    winner_df = winner_df.drop_duplicates(ignore_index=True)

    winners = winner_df['winner'].tolist()
    b = all_team_df['teamname'].isin(winners)
    missing_df = all_team_df[b == False]
    return missing_df

## all winners must be in teamname
def get_ghost_winners(df):
    all_teams = set(df['teamname'].tolist())
    b = df['winner'].isin(all_teams)
    ghost_winners_df = df[winner_columns].loc[b == False]
    return ghost_winners_df

# 
def input_player_mapstat(filename):
    df = pd.read_csv(filename)
    df['winner'] = list(map(lambda x: x.replace('%20', ' '), df['winner']))
#    ghost_df = get_ghost_winners(df)
#    if ghost_df.shape[0]:
#        print(ghost_df)
#    else:
#    append_team_map(df)
    return df

In [105]:
df = input_player_mapstat('player_mapstat_20200617_20200610.csv')

,player_id,player,match_id,date,kills,hs,assists,flash_assists,kdratio,deaths,...,adr,fkdiff,rating,teamname,winner,winner_id,total_rounds,mapstatid,win_rounds,map
16597,16597,4pack,2342227,2020-06-17,31,9,7,2,7,20,...,106,4,1.56,"blood, sweat, and tears","blood, sweat, and tears",10692,30.0,104371,16,Inferno
9572,9572,Marved,2342227,2020-06-17,19,11,9,1,9,20,...,81,-2,1.08,"blood, sweat, and tears","blood, sweat, and tears",10692,30.0,104371,16,Inferno
14131,14131,retchy,2342227,2020-06-17,19,13,2,1,2,19,...,64,-2,1.01,"blood, sweat, and tears","blood, sweat, and tears",10692,30.0,104371,16,Inferno


In [126]:
file = input_files[1]
df = input_player_mapstat(file)

             winner  winner_id
12092  Paquet%C3%A1      10669
555    Paquet%C3%A1      10669
19869  Paquet%C3%A1      10669
14736  Paquet%C3%A1      10669
9218   Paquet%C3%A1      10669
18835  Paquet%C3%A1      10669
10790  Paquet%C3%A1      10669
15449  Paquet%C3%A1      10669
18836  Paquet%C3%A1      10669
10563  Paquet%C3%A1      10669


In [128]:
df.loc[[12092, 555, 10563]]

,player_id,player,match_id,date,kills,hs,assists,flash_assists,kdratio,deaths,...,adr,fkdiff,rating,teamname,winner,winner_id,total_rounds,mapstatid,win_rounds,map
12092,12092,max,2341783,2020-05-29,17,7,6,2,6,18,...,71,2,1.00,9z,Paquet%C3%A1,10669,25.0,103358,16,Inferno
555,555,bit,2341783,2020-05-29,16,5,3,1,3,18,...,73,1,0.93,9z,Paquet%C3%A1,10669,25.0,103358,16,Inferno
10563,10563,caike,2341783,2020-05-29,13,5,6,2,6,16,...,55,-1,0.98,Paquetá,Paquet%C3%A1,10669,25.0,103358,16,Inferno


## assemble files

In [42]:
input_files = [
 'player_mapstat_20191231_20190716.csv',
 'player_mapstat_20191231_20191215.csv',
 'player_mapstat_20191215_20191201.csv',
 'player_mapstat_20191205_20191201.csv',
 'player_mapstat_20191201_20191115.csv',
 'player_mapstat_20191115_20191101.csv',
 'player_mapstat_20191101_20191015.csv',
 'player_mapstat_20191015_20191001.csv',
 'player_mapstat_20191001_20190915.csv',
 'player_mapstat_20190915_20190901.csv',
 'player_mapstat_20190901_20190815.csv',
 'player_mapstat_20190815_20190801.csv',
 'player_mapstat_20190731_20190715.csv',
 'player_mapstat_20190716_20190701.csv',
 'player_mapstat_20190701_20190104.csv',
 'player_mapstat_20190701_20190615.csv',
 'player_mapstat_20190615_20190601.csv',
 'player_mapstat_20190601_20190515.csv',
 'player_mapstat_20190515_20190501.csv',
 'player_mapstat_20190501_20190415.csv',
 'player_mapstat_20190417_20190401.csv',
 'player_mapstat_20190402_20190315.csv',
 'player_mapstat_20190317_20190301.csv',
 'player_mapstat_20190301_20190215.csv',
 'player_mapstat_20190215_20190201.csv',
 'player_mapstat_20190201_20190115.csv',
 'player_mapstat_20190116_20190104.csv',
]

In [43]:
first = True
ddf = None
start_date = None
for file in input_files:
    df = input_player_mapstat(file)
    size = df.shape[0]
    start = df[['date']].iloc[0]
    if first:
        start_date = start
    end = df[['date']].iloc[size -1]
    s_df = df.drop_duplicates()
    print(f'{start[0]} {end[0]}  {df.shape[0]} {s_df.shape[0]} {file}')
    if first:
        ddf = s_df
        first = False
    else:
        ddf = pd.concat([ddf, s_df])
ddf = ddf.drop_duplicates()

2019-12-31 2019-07-01  78694 76353 player_mapstat_20191231_20190716.csv
2019-12-31 2019-12-15  9950 3864 player_mapstat_20191231_20191215.csv
2019-12-15 2019-12-04  15713 7132 player_mapstat_20191215_20191201.csv
2019-12-05 2019-12-01  4002 1601 player_mapstat_20191205_20191201.csv
2019-12-01 2019-11-15  18923 7909 player_mapstat_20191201_20191115.csv
2019-11-15 2019-11-01  14321 6821 player_mapstat_20191115_20191101.csv
2019-11-01 2019-10-15  16857 8234 player_mapstat_20191101_20191015.csv
2019-10-15 2019-10-01  14759 6595 player_mapstat_20191015_20191001.csv
2019-10-01 2019-09-15  22194 9853 player_mapstat_20191001_20190915.csv
2019-09-15 2019-09-01  15402 6761 player_mapstat_20190915_20190901.csv
2019-09-01 2019-08-15  12372 5191 player_mapstat_20190901_20190815.csv
2019-08-15 2019-08-01  9100 3950 player_mapstat_20190815_20190801.csv
2019-07-31 2019-07-15  12851 5751 player_mapstat_20190731_20190715.csv
2019-07-16 2019-07-01  11215 5032 player_mapstat_20190716_20190701.csv
2019-07-

In [44]:
output_filename = 'player_mapstat_2019.csv'
output_filename

'player_mapstat_2019.csv'

In [45]:
ddf.to_csv(output_filename, index=False)

In [189]:
file = 'player_mapstat_2019.csv.zip'

In [46]:
df = input_player_mapstat(file)
df.head(3)

,player_id,player,match_id,date,kills,hs,assists,flash_assists,kdratio,deaths,...,adr,fkdiff,rating,teamname,winner,winner_id,total_rounds,mapstatid,win_rounds,map
2555,2555,yam,2330235,2019-01-16,18,3,1,1,1,14,...,76,2,1.10,ex-Space Soldiers,ENCE,4869,22.0,80194,16,Train
9353,9353,paz,2330235,2019-01-16,12,6,2,0,2,16,...,63,-1,0.82,ex-Space Soldiers,ENCE,4869,22.0,80194,16,Train
8575,8575,Calyx,2330235,2019-01-16,9,5,1,0,1,15,...,49,1,0.71,ex-Space Soldiers,ENCE,4869,22.0,80194,16,Train


In [47]:
df.shape

(5455, 21)

In [177]:
d1 = ddf.iloc[0:120]
d2 = d1.drop_duplicates()
print(f'{d1.shape[0]}, {d2.shape[0]}')

120, 100


In [69]:
b = ddf['winner'].isin(all_teams)
b

16597    False
9572     False
14131    False
14109    False
16649    False
         ...  
2553     False
3669     False
8242     False
15204    False
8323     False
Name: winner, Length: 17158, dtype: bool

In [72]:
ghost_winners_df = ddf[winner_columns].loc[b == False]
if ghost_winners_df.shape[0]:
    print(ghost_winners_df)

                        winner  winner_id
16597  blood, sweat, and tears      10692
9572   blood, sweat, and tears      10692
14131  blood, sweat, and tears      10692
14109  blood, sweat, and tears      10692
16649  blood, sweat, and tears      10692
...                        ...        ...
2553                 Illuminar       8813
3669                 Illuminar       8813
8242                 Illuminar       8813
15204                Illuminar       8813
8323                 Illuminar       8813

[17158 rows x 2 columns]


In [71]:
ddf[winner_columns]

,winner,winner_id
16597,"blood, sweat, and tears",10692
9572,"blood, sweat, and tears",10692
14131,"blood, sweat, and tears",10692
14109,"blood, sweat, and tears",10692
16649,"blood, sweat, and tears",10692
...,...,...
2553,Illuminar,8813
3669,Illuminar,8813
8242,Illuminar,8813
15204,Illuminar,8813


In [63]:
ghost_winner_df = check_winner(ddf)
ghost_winner_df

In [28]:
# check team without id:
winner_df = pd.read_csv(winner_filename)
winner_df = winner_df.drop_duplicates(ignore_index=True)
winner_df

,winner,winner_id
0,"blood, sweat, and tears",10692
1,Keep the Comms Up,10691
2,TeamOne,6947
3,AA,10676
4,Polar Ace,9497
...,...,...
136,BIG Academy,10254
137,Tenerife Titans,10162
138,9INE,10278
139,Warriors International,10719


In [40]:
all_team_df = pd.read_csv(all_team_filename)
all_team_df = all_team_df.drop_duplicates(ignore_index=True)
all_team_df

,teamname
0,"blood, sweat, and tears"
1,Levitate
2,Keep the Comms Up
3,Infinity
4,TeamOne
...,...
169,Morning Light
170,Under 21
171,Big Frames
172,Generation of Miracles


In [54]:
winners = winner_df['winner'].tolist()
b = all_team_df['teamname'].isin(winners)
missing_df = all_team_df[b == False]
missing_df

,teamname
20,pro100
64,Aalborg Rebels
73,Vintage
96,Rhein Neckar
99,Izako Boars
105,WiLD
106,ajuri
107,ECLOT
108,SJ
114,RED Canids


In [30]:
team_map_df = pd.read_csv(all_team_filename)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 176, saw 2


In [29]:
team_map_df = pd.read_csv(all_team_filename)
team_map_df = team_map_df.drop_duplicates(ignore_index=True)
team_map_df

ParserError: Error tokenizing data. C error: Expected 1 fields in line 176, saw 2


In [19]:
all_teams = ddf[['teamname']].drop_duplicates()
all_teams

,teamname
16597,"blood, sweat, and tears"
16647,Levitate
14613,Keep the Comms Up
11524,Infinity
17585,TeamOne
...,...
9884,Morning Light
16902,Under 21
11948,Big Frames
18032,Generation of Miracles


In [15]:
all_teams_df = pd.DataFrame(data={"teamname": list(all_teams)})
all_teams_df

,teamname
0,GODSENT
1,Baecon
2,HOLLYWOOD
3,Morning Light
4,CLEANTmix
...,...
169,Oceanus
170,Heretics
171,Aalborg Rebels
172,Heroic


In [16]:
all_teams_df.to_csv("all_teams.csv", index=False)

In [17]:
all_teams_df = pd.read_csv('all_teams.csv')
all_teams_df

,teamname
0,GODSENT
1,Baecon
2,HOLLYWOOD
3,Morning Light
4,CLEANTmix
...,...
169,Oceanus
170,Heretics
171,Aalborg Rebels
172,Heroic


In [25]:
def clean_file(df_input):
    df = df_input.copy()
    df['winner'] = list(map(lambda x: x.replace('%20', ' '), df['winner']))

16597    blood, sweat, and tears
9572     blood, sweat, and tears
14131    blood, sweat, and tears
14109    blood, sweat, and tears
16649    blood, sweat, and tears
                  ...           
2553                   Illuminar
3669                   Illuminar
8242                   Illuminar
15204                  Illuminar
8323                   Illuminar
Name: winner, Length: 17158, dtype: object

In [ ]:
for idx
    if '%20' in winner:
        # replace(old, new, count)
        new_str = winner.replace('%20', ' ')
        print(f'{winner} => {new_str}')


In [35]:
teamnames = ddf['teamname']

In [36]:
for winner in df['winner']:
    if winner not in ddf['teamname']:
        print(f"winner '{winner}'")


winner 'blood, sweat, and tears'
winner 'blood, sweat, and tears'
winner 'blood, sweat, and tears'
winner 'blood, sweat, and tears'
winner 'blood, sweat, and tears'
winner 'blood, sweat, and tears'
winner 'blood, sweat, and tears'
winner 'blood, sweat, and tears'
winner 'blood, sweat, and tears'
winner 'blood, sweat, and tears'
winner 'Keep the Comms Up'
winner 'Keep the Comms Up'
winner 'Keep the Comms Up'
winner 'Keep the Comms Up'
winner 'Keep the Comms Up'
winner 'Keep the Comms Up'
winner 'Keep the Comms Up'
winner 'Keep the Comms Up'
winner 'Keep the Comms Up'
winner 'Keep the Comms Up'
winner 'TeamOne'
winner 'TeamOne'
winner 'TeamOne'
winner 'TeamOne'
winner 'TeamOne'
winner 'TeamOne'
winner 'TeamOne'
winner 'TeamOne'
winner 'TeamOne'
winner 'TeamOne'
winner 'AA'
winner 'AA'
winner 'AA'
winner 'AA'
winner 'AA'
winner 'AA'
winner 'AA'
winner 'AA'
winner 'AA'
winner 'AA'
winner 'Polar Ace'
winner 'Polar Ace'
winner 'Polar Ace'
winner 'Polar Ace'
winner 'Polar Ace'
winner 'Polar A

winner 'Apeks'
winner 'Apeks'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Apeks'
winner 'Apeks'
winner 'Apeks'
winner 'Apeks'
winner 'Apeks'
winner 'Apeks'
winner 'Apeks'
winner 'Apeks'
winner 'Apeks'
winner 'Apeks'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Unicorns of Love'
winner 'Apeks'
winner 'Apeks'
winner 'Ap

winner 'PACT'
winner 'PACT'
winner 'PACT'
winner 'PACT'
winner 'PACT'
winner 'PACT'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'HAVU'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winner 'Ninjas To Be'
winn

winner 'mens'
winner 'mens'
winner 'mens'
winner 'mens'
winner 'mens'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'forZe'
winner 'Sp

winner 'Paradox'
winner 'Paradox'
winner 'Paradox'
winner 'Paradox'
winner 'Paradox'
winner 'Paradox'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'SKADE'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX'
winner 'ALTERNATE aTTaX

winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'TYLOO'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winner 'ViCi'
winne

winner 'forZe'
winner 'ENCE'
winner 'ENCE'
winner 'ENCE'
winner 'ENCE'
winner 'ENCE'
winner 'ENCE'
winner 'ENCE'
winner 'ENCE'
winner 'ENCE'
winner 'ENCE'
winner 'HellRaisers'
winner 'HellRaisers'
winner 'HellRaisers'
winner 'HellRaisers'
winner 'HellRaisers'
winner 'HellRaisers'
winner 'HellRaisers'
winner 'HellRaisers'
winner 'HellRaisers'
winner 'HellRaisers'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Sprout'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 'Heretics'
winner 

winner 'CR4ZY'
winner 'CR4ZY'
winner 'CR4ZY'
winner 'CR4ZY'
winner 'CR4ZY'
winner 'CR4ZY'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'AVEZ'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground Zero'
winner 'Ground

In [40]:
teamname = teamnames.iloc[0]
teamname

'blood, sweat, and tears'

In [42]:
if teamname not in teamnames:
    print(f"teamname '{teamname}'")

teamname 'blood, sweat, and tears'


In [43]:
ddf.shape[0]

17158

In [44]:
winners = df['winner'].count()

In [45]:
winners

17158

In [68]:
team_name_id_map = {}
for i in range(ddf.shape[0]):
    name = df['winner'].iloc[i]
    team_name_id_map[name] = int(df['winner_id'].iloc[i])
for i in range(ddf.shape[0]):
    name = df['winner'].iloc[i]
    id_ = df['winner_id'].iloc[i]
    if team_name_id_map[name] != id_:
        print(f'index={i}, name={name}, id={id_}')

In [51]:
losers = []
for i in range(ddf.shape[0]):
    name = df['teamname'].iloc[i]
    if name not in team_name_id_map.keys():
        if name not in losers:
            losers.append(name)
            print(f'index={i}, name={name}')
losers

index=320, name=pro100
index=1960, name=Aalborg Rebels
index=2360, name=Vintage
index=4617, name=Rhein Neckar
index=4652, name=Izako Boars
index=4977, name=WiLD
index=5232, name=ajuri
index=5277, name=ECLOT
index=5482, name=SJ
index=6707, name=RED Canids
index=6797, name=Sinisters
index=7952, name=PC419
index=8177, name=Forbidden
index=8182, name=Control
index=8272, name=really weird
index=8327, name=Buffdaddy's Paypal
index=8517, name=Hawks
index=8567, name=Baecon
index=9867, name=Quantum Bellator Fire
index=10567, name=ETHEREAL
index=10652, name=ztrewq
index=10662, name=Unique
index=10677, name=FTW
index=12206, name=Asterius
index=12651, name=RAMS
index=13511, name=Redemption POA
index=13696, name=CLEANTmix
index=13981, name=Lucid Dream
index=14926, name=Elites
index=15671, name=Morning Light
index=16423, name=Big Frames
index=17038, name=Generation of Miracles
index=17083, name=Juggernauts


['pro100',
 'Aalborg Rebels',
 'Vintage',
 'Rhein Neckar',
 'Izako Boars',
 'WiLD',
 'ajuri',
 'ECLOT',
 'SJ',
 'RED Canids',
 'Sinisters',
 'PC419',
 'Forbidden',
 'Control',
 'really weird',
 "Buffdaddy's Paypal",
 'Hawks',
 'Baecon',
 'Quantum Bellator Fire',
 'ETHEREAL',
 'ztrewq',
 'Unique',
 'FTW',
 'Asterius',
 'RAMS',
 'Redemption POA',
 'CLEANTmix',
 'Lucid Dream',
 'Elites',
 'Morning Light',
 'Big Frames',
 'Generation of Miracles',
 'Juggernauts']

In [53]:
team_name_id_map

{'blood, sweat, and tears': 10692,
 'Keep the Comms Up': 10691,
 'TeamOne': 6947,
 'AA': 10676,
 'Polar Ace': 9497,
 'MIBR': 9215,
 'Syman': 8772,
 'CR4ZY': 10150,
 'Ambush': 7674,
 'Lyngby Vikings': 8963,
 'Nordavind': 8769,
 'sAw': 10567,
 'Endpoint': 7234,
 'Complexity': 5005,
 'OG': 10503,
 'FATE': 9863,
 'Singularity': 6978,
 'forZe': 8135,
 'HellRaisers': 5310,
 'Virtus.pro': 5378,
 'GODSENT': 6902,
 'AGO': 8068,
 'AGF': 8704,
 'GamerLegion': 9928,
 'Sprout': 8637,
 'Natus Vincere': 4608,
 'Espada': 8669,
 'Spirit': 7020,
 'SKADE': 10386,
 'Gambit Youngsters': 9976,
 'ORDER': 8668,
 'Hard4U': 10573,
 'Renegades': 6211,
 'Nemiga': 7969,
 'Moscow Five Academy': 10627,
 'Thunder Logic': 9615,
 'New England Whalers': 9888,
 'Heroic': 7175,
 'HAVU': 7865,
 'North': 7533,
 'Sigma': 10372,
 'Ignite': 8706,
 'FaZe': 6667,
 'ENCE': 4869,
 'BIG': 7532,
 'Vitality': 9565,
 'Unicorns of Love': 9812,
 'Apeks': 9806,
 'PACT': 8248,
 'AVANT': 5293,
 'Chiefs': 6010,
 'NiP': 4411,
 'MAD Lions': 8

In [89]:
team_df = pd.read_json(json.dumps(team_name_id_map), orient='index')
team_df

,0
"blood, sweat, and tears",10692
Keep the Comms Up,10691
TeamOne,6947
AA,10676
Polar Ace,9497
...,...
BIG Academy,10254
Tenerife Titans,10162
9INE,10278
Warriors International,10719


In [91]:
team_df = team_df.reset_index()
team_df

,index,0
0,"blood, sweat, and tears",10692
1,Keep the Comms Up,10691
2,TeamOne,6947
3,AA,10676
4,Polar Ace,9497
...,...,...
136,BIG Academy,10254
137,Tenerife Titans,10162
138,9INE,10278
139,Warriors International,10719


In [92]:
team_df.columns = ['team_name', 'team_id']
team_df

,team_name,team_id
0,"blood, sweat, and tears",10692
1,Keep the Comms Up,10691
2,TeamOne,6947
3,AA,10676
4,Polar Ace,9497
...,...,...
136,BIG Academy,10254
137,Tenerife Titans,10162
138,9INE,10278
139,Warriors International,10719


In [93]:
team_filename = 'team_id_name.csv'
team_df.to_csv(team_filename, index=False)

In [94]:
team_ddf = pd.read_csv(team_filename)
team_ddf

,team_name,team_id
0,"blood, sweat, and tears",10692
1,Keep the Comms Up,10691
2,TeamOne,6947
3,AA,10676
4,Polar Ace,9497
...,...,...
136,BIG Academy,10254
137,Tenerife Titans,10162
138,9INE,10278
139,Warriors International,10719


In [85]:
a = 'someone' in team_ddf['team_name'].values
print(a)

False


In [84]:
a = 'blood, sweat, and tears' in team_ddf['team_name'].values
print(a)

True


In [ ]:
df.append(df2, ignore_index=True)